In [1]:
# Core
import numpy as np
import pandas as pd

In [2]:
# Local imports
import kepler_sieve
from asteroid_direction import ast, c, mpd, dpm, prep_ast_block, calc_dir_ast_block
from asteroid_spline import make_spline_ast_dir
from sky_patch import dir2SkyPatchID
from db_utils import sp2df, df2db

# Build block of asteroid directions at high resolution

In [3]:
# Time range for asteroid directions
mjd0: int = 58000
mjd1: int = 60000
# interval_min: int = 1
interval_min: int = 5

# Asteroid block
n0: int = 1
n1: int = 11

In [4]:
# Prepare asteroid block
t_obs, asteroid_id = prep_ast_block(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1, interval_min=interval_min)

In [5]:
# Build spline of asteroid direction
spline_u = make_spline_ast_dir(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1)

In [6]:
# Calculate asteroid directions from position and velocity
u_ast, light_time = spline_u(t_obs, asteroid_id)

In [7]:
# The TimeID
time_id = np.round(t_obs * mpd).astype(np.int64)

In [8]:
# Calculate the SkyPatchID
N_sp = 1024
sky_patch_id = dir2SkyPatchID(dir=u_ast, N=N_sp)

In [9]:
# Calculate the asteroid number in this batch from asteroid_id
asteroid_id_unq = np.unique(asteroid_id)
N_ast = asteroid_id_unq.shape[0]
ast_bn_unq = np.arange(N_ast, dtype=np.int)
ast_bn = np.searchsorted(asteroid_id_unq, asteroid_id)

In [10]:
# ast_bn

In [11]:
# Number of rows
N_row = asteroid_id.shape[0]

In [12]:
# Rows with a new asteroid
is_new_ast = np.ones(N_row, dtype=np.bool)
is_new_ast[1:N_row] = (asteroid_id[1:N_row] != asteroid_id[0:N_row-1])
idx_new_ast = np.argwhere(is_new_ast).flatten()

In [13]:
# Rows with a new sky patch entry; includes row with a new asteroid
is_new_skypatch = np.ones(N_row, dtype=np.bool)
is_new_skypatch[1:N_row] = (sky_patch_id[1:N_row] != sky_patch_id[0:N_row-1])

In [14]:
# Flag for last segment
is_last_segment = np.zeros(N_row, dtype=np.bool)
is_last_segment[idx_new_ast-1] = True

In [15]:
# Combined mask for interior segments
mask_all = (is_new_ast | is_new_skypatch)
mask_out = mask_all & (~is_last_segment)

In [16]:
# Row numbers for these masks
idx_all = np.argwhere(mask_all).flatten()
idx_out = np.argwhere(mask_out).flatten()

In [17]:
# The segment number, increasing across all asteroids in batch
sn_cum = np.cumsum(mask_all)
sn_base = sn_cum[is_new_ast]
sn = sn_cum - sn_base[ast_bn]

In [22]:
np.sum(is_last_segment)

10

In [24]:
np.sum(mask_out)

156665

In [28]:
# Wrap into DataFrame
df_tbl = {
    'AsteroidID': asteroid_id,
    'Segment': sn,
    'TimeID': time_id,
    'SkyPatchID': sky_patch_id,
    'IsOutput': mask_out,
}

df_all = pd.DataFrame(df_tbl)

In [29]:
df_all

,AsteroidID,Segment,TimeID,SkyPatchID,IsOutput
0,1,0,83520000,6382172,True
1,1,0,83520005,6382172,False
2,1,0,83520010,6382172,False
3,1,0,83520015,6382172,False
4,1,0,83520020,6382172,False
...,...,...,...,...,...
5760005,10,11744,86399980,18877959,True
5760006,10,11744,86399985,18877959,False
5760007,10,11744,86399990,18877959,False
5760008,10,11744,86399995,18877959,False


In [30]:
# Create DataFrame for just the rows that start interior segments
cols = ['AsteroidID', 'Segment', 'SkyPatchID', 'TimeID', 'IsOutput']
df = df_all.loc[mask_all, cols]

In [31]:
df

,AsteroidID,Segment,SkyPatchID,TimeID,IsOutput
0,1,0,6382172,83520000,True
27,1,1,6382171,83520135,True
45,1,2,6384219,83520225,True
63,1,3,6384218,83520315,True
99,1,4,6384217,83520495,True
...,...,...,...,...,...
5759870,10,11740,18877955,86399305,True
5759904,10,11741,18877956,86399475,True
5759938,10,11742,18877957,86399645,True
5759972,10,11743,18877958,86399815,True


In [25]:
idx_all

array([      0,      27,      45, ..., 5759938, 5759972, 5760005])

In [26]:
idx_all.shape

(156665,)

In [37]:
idx_next = np.concatenate([idx_all[1:], idx_all[0:1]])
idx_next

array([     27,      45,      63, ..., 5759972, 5760005,       0])

In [38]:
# TimeID of each segment
time_id_0 = time_id[idx_all]
time_id_1 = time_id[idx_next]

In [39]:
df['TimeID_0'] = time_id_0
df['TimeID_1'] = time_id_1

In [44]:
# Mask down to interior segments only
df = df[mask_out]

ValueError: Item wrong length 5760010 instead of 156665.

In [40]:
df

,AsteroidID,Segment,SkyPatchID,TimeID,TimeID_0,TimeID_1
0,1,0,6382172,83520000,83520000,83520135
27,1,1,6382171,83520135,83520135,83520225
45,1,2,6384219,83520225,83520225,83520315
63,1,3,6384218,83520315,83520315,83520495
99,1,4,6384217,83520495,83520495,83520675
...,...,...,...,...,...,...
5759870,10,11740,18877955,86399305,86399305,86399475
5759904,10,11741,18877956,86399475,86399475,86399645
5759938,10,11742,18877957,86399645,86399645,86399815
5759972,10,11743,18877958,86399815,86399815,86399980


In [41]:
dt = (df.TimeID_1 - df.TimeID_0).values

In [42]:
np.min(dt)

-2879990

In [43]:
np.max(dt)

10870